In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
import math
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics.pairwise import linear_kernel

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
path="D:/scrapping/magicbricks/"
df=pd.read_csv(f'{path}data_gnr_feb.csv')

In [3]:
df.head()

,name,id,createdDate,description,Society,url,price,priceInWord,cityName,addressLocality,...,floorno,furnshingstatus,agentName,agentCompanyName,agentMaskedmobilenumber,projectName,builderName,propertyType,locality,postedBy
0,2 BHK Apartment for Sale in Golfforeste AC Apa...,domcache_srp_46630105,20200211,"2 Bath,Unfurnished,16 floor Paramount Golf For...",Golfforeste AC Apartments,https://www.magicbricks.com/propertyDetails/2-...,3710000.0,37.1 Lac,Greater Noida,Zeta 1,...,16,Unfurnished,Praveen Choudhary,BD Green Home Pvt. Ltd.,+91-88XXXXXXX90,Golfforeste AC Apartments,Paramount Group,Apartment,Zeta 1,Agent
1,4 BHK Apartment for Sale in Purvanchal Royal C...,domcache_srp_46527183,20200131,"4 Bath,Semi-Furnished,19 floor A life where yo...",Purvanchal Royal City,https://www.magicbricks.com/propertyDetails/4-...,11200000.0,1.12 Cr,Greater Noida,Chi 5,...,19,Semi-Furnished,Shashi Kapoor,Purvanchal Projects Pvt. Ltd.,+91-78XXXXXXX42,Purvanchal Royal City,Purvanchal Projects Pvt. Ltd.,Apartment,Chi 5,Builder
2,"3 BHK Apartment for Sale in SKA Metro Ville, E...",domcache_srp_46365097,20200130,"2 Bath,Semi-Furnished SKA Metro Ville is the f...",SKA Metro Ville,https://www.magicbricks.com/propertyDetails/3-...,3430000.0,34.3 Lac,Greater Noida,"Eta 2, Eta",...,NaN,Semi-Furnished,SKA GROUP,NaN,+91-92XXXXXXX15,SKA Metro Ville,SKA Group,Apartment,"Eta 2, Eta",Builder
3,"2 BHK Apartment for Sale in Arihant Ambar, Gre...",domcache_srp_34690341,20200207,"2 Bath,Semi-Furnished,20 floor Arihant Ambar i...",Arihant Ambar,https://www.magicbricks.com/propertyDetails/2-...,4300000.0,43 Lac,Greater Noida,Greater Noida West,...,20,Semi-Furnished,Rajat Verma,Arihant Group,+91-88XXXXXXX26,Arihant Ambar,Arihant Group,Apartment,Greater Noida West,Builder
4,"1 BHK Apartment for Sale in Prideville , Yamun...",domcache_srp_45300261,20200210,"1 Bath,Semi-Furnished We at Prideville underst...",Prideville,https://www.magicbricks.com/propertyDetails/1-...,3700000.0,37 Lac,Greater Noida,Yamuna Expressway,...,NaN,Semi-Furnished,Vipin,Imperia Structures Ltd.,+91-74XXXXXXX93,Prideville,Imperia Structures Ltd.,Apartment,Yamuna Expressway,Agent


In [4]:
df.propertyType.unique()

array(['Apartment', 'Villa', 'Penthouse', 'Studio Apartment',
       'Builder Floor Apartment'], dtype=object)

In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
url_symbol=re.compile('[^https?:\/\/.*[\r\n]*]')
STOPWORDS = set(stopwords.words('english'))

In [6]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text =url_symbol.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [7]:
df[df.Society=='Nirala Aspire'].T

,1526
name,"3 BHK Apartment for Sale in Nirala Aspire, Gre..."
id,domcache_srp_35577511
createdDate,20200207
description,"3 Bath,Unfurnished,South - East facing Nirala ..."
Society,Nirala Aspire
url,https://www.magicbricks.com/propertyDetails/3-...
price,5.60538e+06
priceInWord,56.1 Lac
cityName,Greater Noida
addressLocality,Greater Noida West


In [8]:
property_type=['Apartment', 'Villa', 'Penthouse', 'Studio-Apartment','Builder-Floor-Apartment']

In [9]:
property_type=[p.lower() for p in property_type]

In [10]:
property_type

['apartment',
 'villa',
 'penthouse',
 'studio-apartment',
 'builder-floor-apartment']

In [11]:
df.fillna('',inplace=True)

In [12]:
property_tags=[]

In [13]:
city_master=df.cityName.unique()

In [14]:
city_master=[c.lower() for c in city_master]

In [15]:
locality_master=df.locality.unique()

In [16]:
locality_master=[l.lower() for l in locality_master]

In [17]:
df.Society.fillna('',inplace=True)

In [18]:
locality_master

['zeta 1',
 'chi 5',
 'eta 2, eta',
 'greater noida west',
 'yamuna expressway',
 'jaypee greens',
 'sector 27',
 'zeta 2',
 'pari chowk',
 'omicron 3',
 'chi 4',
 'sector 10',
 'phi 4',
 'iteda, amrapali dream valley',
 'chuharpur khadar, sector-chi',
 'tech zone, gautam budh nagar, yamuna expressway',
 'omicron 1',
 'ecotech 3',
 'techzone 4',
 'sector-4',
 'knowledge park 3',
 'sector 1',
 'anand ashray, phi 3',
 'noida extention',
 'sector-phi',
 'sector 2',
 'pi 2',
 'omicron',
 'noida extension',
 'beta 2',
 'unitech horizon, sector-pi',
 'omega 1',
 'sigma 4',
 'sector-phi1',
 'sector mu',
 '',
 'sector-pi',
 'alpha 1',
 'surajpur',
 'tilpata karanwas',
 'omicron 2',
 'dadri',
 'cassia fistula estate, sector-chi',
 'gautam budh nagar',
 'dadri main road',
 'jitendra nagar, achheja',
 'eldeco green meadows, sector-pi',
 'aimnabad',
 'chi 2',
 'surajpur site -c industrial',
 'cassia sigma, sigma 3',
 'sector-zeta',
 'phi 2',
 'upsidc site c block h',
 'jaypee greens golf course, n

In [19]:
society_master=df.Society.unique()

In [20]:
society_master=[s.lower() for s in society_master]

In [21]:
def get_lanmarksearch(df,landmark):
    df_result=pd.DataFrame()
    #lanmark=landmark.lower()
    if(landmark in city_master):
        df_result=df[df['cityName'].str.lower()==landmark]
        print("cityName")
    elif(landmark in locality_master):
        print("lacality")
        df_result=df[df['locality'].str.lower()==landmark]
    elif(landmark in society_master):
        print("Society")
        df_result=df[df['Society'].str.lower()==landmark]
    else:
        df_result=df
    #print(df_result.shape)
    return df_result

In [22]:
def getPropertytype(df_r,ptypes):
    #if ptype in property_type:
    df_r=df_r[df_r['propertyType'].str.lower().isin([x.lower() for x in ptypes])]
    print(df_r.shape)
    return df_r

In [23]:
def getinBudget(df_r,minimum=100000,maximum=999999999):
    minimum=int(minimum)
    maximum=int(maximum)
    df_r=df_r[(df_r.price<=maximum) & (df_r.price>=minimum)]
    return df_r
    

In [24]:
def roomnumberfilter(df_r,minimum=1,maximum=10):
    minimum=int(minimum)
    maximum=int(maximum)
    df_r=df_r[(df_r.bedroom<=maximum) & (df_r.bedroom>=minimum)]
    return df_r

In [25]:
def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

In [26]:
wnl = WordNetLemmatizer()

In [72]:
def lemmatize(x):
    x = ' '.join(word for word in x.split(' ') if word not in STOPWORDS)
    #print(type(s))
    text=x.rsplit(" ")
    s = [wnl.lemmatize(word) for word in text]
    text = ' '.join(word for word in s)
    return text

In [28]:
df = df.assign(lemma_desc = df.description.apply(lambda x: lemmatize(x)))
#df_res['lemma_desc']=df['description'].apply(lemmatize_text)

In [29]:
def generateNGram(text,n=2):
    text=text.lower()
    tokens=text.split(" ")
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return ["_".join(ngram) for ngram in ngrams]

In [30]:
def get_index(reqbased,features):
    #print((features))
    features = ' '.join(word for word in features.split() if word not in STOPWORDS)
    features=lemmatize(features)
    score_ind=0
    index_ind=0
    one_gm_score=0
    two_gm_score=0
    three_gm_score=0
    for index,row  in reqbased.iterrows():
        desc=row['lemma_desc']
        score=0
        for i in range(1,3):
            #print(index)
            #print(re.sub(r'[^\w\s]',' ',features))
            #print(type(generateNGram(desc,i)),type(generateNGram(features,i)))
            
            score = score+ counter_cosine_similarity(Counter(list(generateNGram(desc,i))), Counter(list(generateNGram(features,i))))
            #print(score)
        #score=np.divide(score,2)
        if score>score_ind:
            score_ind=score
            index_ind=index
    print(index_ind)
    print(score_ind)
    return reqbased[reqbased.index==index_ind].index[0]  

In [31]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [32]:
def cosinesimilarity(df_res):
    tfidf_matrix = tf.fit_transform(df_res['lemma_desc'])
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    return cosine_similarities

In [33]:
def get_recommedation_budget(df_res,features):
    recommended_properties = []
    df_result=pd.DataFrame(columns=df_res.columns)
    df_res['lemma_desc']=df_res['lemma_desc'].apply(clean_text)
    cosine_similarities=cosinesimilarity(df_res)
    indices = pd.Series(df_res.index)
    features=clean_text(features)
    property_id=get_index(df_res,features)
    df_result=df[df.id==property_id]
    idx=indices[indices == property_id].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    top_20_indexes = list(score_series.iloc[0:20].index)
    #print(top_20_indexes)
    # populating the list with the names of the top 10 matching hotels
    for i in top_20_indexes:
        name=list(df.index)[i]
        #print(df[df.index == name])
        df_result=pd.concat([df_result,df[df.index==name]],axis=0)
        recommended_properties.append(list(df.index)[i])
    
        
    return recommended_properties,df_result

In [34]:
features="I want 2 BHK with study apartment with 1400 sqft between 6 to 12 floor in north-east facing in Greater Noida West"

In [35]:
#df_res[df_res.index=="domcache_srp_27982777"]

NameError: name 'df_res' is not defined

In [36]:
def recommandation_top(landmark,ptypes,min_budget,max_budget,min_room,max_room,features):
    landmark=landmark.lower()
    #ptype=ptype.lower()
    features=features.lower()
    min_budget=int(min_budget)
    max_budget=int(max_budget)
    min_room=int(min_room)
    max_room=int(max_room)
    features=features.lower()
    df_res=get_lanmarksearch(df,landmark)
    #print(df_res.locality.unique())
    df_res=getinBudget(df_res,min_budget,max_budget)
    df_res=getPropertytype(df_res, ptypes)
    df_res=roomnumberfilter(df_res,min_room,max_room)
    recom_id,df_result=get_recommedation_budget(df_res,features)
    return df_result
    
    

In [37]:
recommandation_top("Noida Extension",['Apartment','Penthouse'],1000000,6000000,2,3,"I want 2 to 3 BHK with study apartment with min 1400 sqft between 6 to 12 floor in north-east facing in Greater Noida West")

lacality
(31, 27)
200
0.5665988205752792


,name,id,createdDate,description,Society,url,price,priceInWord,cityName,addressLocality,...,furnshingstatus,agentName,agentCompanyName,agentMaskedmobilenumber,projectName,builderName,propertyType,locality,postedBy,lemma_desc
3,"2 BHK Apartment for Sale in Arihant Ambar, Gre...",domcache_srp_34690341,20200207,"2 Bath,Semi-Furnished,20 floor Arihant Ambar i...",Arihant Ambar,https://www.magicbricks.com/propertyDetails/2-...,4.3e+06,43 Lac,Greater Noida,Greater Noida West,...,Semi-Furnished,Rajat Verma,Arihant Group,+91-88XXXXXXX26,Arihant Ambar,Arihant Group,Apartment,Greater Noida West,Builder,"2 Bath,Semi-Furnished,20 floor Arihant Ambar i..."
26,"3 BHK Apartment for Sale in Gaur Saundaryam, G...",domcache_srp_46953307,20200204,"3 Bath,Semi-Furnished,6 floor,North - East fac...",Gaur Saundaryam,https://www.magicbricks.com/propertyDetails/3-...,8.3e+06,83 Lac,Greater Noida,Greater Noida West,...,Semi-Furnished,Aslam,Mannat Homz,+91-98XXXXXXX33,Gaur Saundaryam,Gaurs Group,Apartment,Greater Noida West,Agent,"3 Bath,Semi-Furnished,6 floor,North - East fac..."
5,"2 BHK Apartment for Sale in Gaur City, Greater...",domcache_srp_40647623,20200210,"2 Bath,Semi-Furnished Gaur City is one of the ...",Gaur City,https://www.magicbricks.com/propertyDetails/2-...,3.88e+06,38.8 Lac,Greater Noida,Greater Noida West,...,Semi-Furnished,Samresh Jha,Gaursons India Pvt. Ltd.,+91-78XXXXXXX06,Gaur City,Gaurs Group,Apartment,Greater Noida West,Builder,"2 Bath,Semi-Furnished Gaur City is one of the ..."
22,"3 BHK Apartment for Sale in Gaur Yamuna City, ...",domcache_srp_45330945,20191206,"2 Bath,Semi-Furnished Gaur Yamuna City (GYC) i...",Gaur Yamuna City,https://www.magicbricks.com/propertyDetails/3-...,3.74e+06,37.4 Lac,Greater Noida,Yamuna Expressway,...,Semi-Furnished,Isha Pandey,Property Plus Realtors,+91-99XXXXXXX48,Gaur Yamuna City,Gaurs Group,Apartment,Yamuna Expressway,Agent,"2 Bath,Semi-Furnished Gaur Yamuna City (GYC) i..."
4,"1 BHK Apartment for Sale in Prideville , Yamun...",domcache_srp_45300261,20200210,"1 Bath,Semi-Furnished We at Prideville underst...",Prideville,https://www.magicbricks.com/propertyDetails/1-...,3.7e+06,37 Lac,Greater Noida,Yamuna Expressway,...,Semi-Furnished,Vipin,Imperia Structures Ltd.,+91-74XXXXXXX93,Prideville,Imperia Structures Ltd.,Apartment,Yamuna Expressway,Agent,"1 Bath,Semi-Furnished We at Prideville underst..."
24,4 BHK Villa for Sale in Rise Resort Residences...,domcache_srp_46523781,20191209,"4 Bath,Semi-Furnished RISE RESORT RESIDENCES i...",Rise Resort Residences,https://www.magicbricks.com/propertyDetails/4-...,1.5e+07,1.50 Cr,Greater Noida,Greater Noida West,...,Semi-Furnished,Sumit Raheja,Wealth Clinic Pvt. Ltd.,+91-92XXXXXXX31,Rise Resort Residences,Rise Group,Villa,Greater Noida West,Agent,"4 Bath,Semi-Furnished RISE RESORT RESIDENCES i..."
25,"4 BHK Apartment for Sale in Meridian Towers, G...",domcache_srp_47324627,20200124,"5 Bath,Unfurnished,23 floor,North - East facin...",Meridian Towers,https://www.magicbricks.com/propertyDetails/4-...,8.676e+06,86.8 Lac,Greater Noida,Greater Noida West,...,Unfurnished,VVIP,Vibhor Vaibhav Infra Home Pvt. Ltd.,+91-85XXXXXXX52,Meridian Towers,VVIP Homes Pvt. Ltd.,Apartment,Greater Noida West,Agent,"5 Bath,Unfurnished,23 floor,North - East facin..."
28,"3 BHK Apartment for Sale in Gaur Saundaryam, G...",domcache_srp_47705747,20200204,"2 Bath,Semi-Furnished,12 floor,East facing 3 B...",Gaur Saundaryam,https://www.magicbricks.com/propertyDetails/3-...,7.4e+06,74 Lac,Greater Noida,Greater Noida West,...,Semi-Furnished,Sachin kumar,,+91-85XXXXXXX45,Gaur Saundaryam,Gaurs Group,Apartment,Greater Noida West,Agent,"2 Bath,Semi-Furnished,12 floor,East facing 3 B..."
21,"2 BHK Apartment for Sale in Sublimis, Greater ...",domcache_srp_47150703,20200211,"2 Bath,Semi-Furnished Sometimes a perfect stor...",Sublimis,https://www.magicbricks.com/propertyDetails/2-...,2.99e+06,29.9 Lac,Greater Noida,Greater Noida West,...,Semi-Furnished,Manoj Chikara,Fix Brixs Infra,+91-98XXXXXXX

In [38]:
#rec,result=get_recommedation_budget(df_res,features)

NameError: name 'df_res' is not defined

In [40]:
def get_propertytype(features):
    #property_type=[]
    for tag in property_type:
        #print(tag)
        pattern = (tag).lower()
        #print(pattern)
        line=features.lower()
        r = re.search(pattern, line, re.IGNORECASE)
        if r:
            #print("inloop")
            pt=re.compile(pattern)
            match=pt.findall(line)
            #print(match)
            property_tags.append(match[0])
    return property_tags
        

In [57]:
flat_list=getnumberofrooms("I want 2 BHK with study apartment with 1400 sqft between 6 to 12 floor in north-east facing in Greater Noida West")

['2']


In [52]:
def keyword_extractor(text):
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    return keywords

In [58]:
import yake

In [53]:
df['keywords'] = df['description'].apply(keyword_extractor)

In [59]:
df['keywords'][4]

[('prideville understand', 0.008509979034576029),
 ('yamuna expressway surrounded', 0.01262089074333973),
 ('personal life', 0.014622568755136666),
 ('yamuna expressway', 0.032464672416855044),
 ('greater noida', 0.036539010072586475),
 ('imperia family', 0.04994466599765292),
 ('lush green environment', 0.058389251428411997),
 ('prideville', 0.0633710666244076),
 ('expressway surrounded', 0.07088690211109176),
 ('professional hubs', 0.0811017516149106),
 ('noida and greater', 0.08328409751731779),
 ('noida', 0.09263299153547212),
 ('professional', 0.10781686940752451),
 ('life', 0.10823354430356685),
 ('projects in sports', 0.11502748482682473),
 ('sports city', 0.11502748482682473),
 ('family', 0.12500208743789157),
 ('understand', 0.1331549369612308),
 ('balance', 0.1331549369612308),
 ('personal', 0.1331549369612308),
 ('surrounded by lush', 0.1376955695957322),
 ('lush green', 0.1376955695957322),
 ('green environment', 0.1376955695957322),
 ('yamuna', 0.1652571977092805),
 ('impe

In [55]:
import spacy

In [56]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc=nlp(df['description'][1])

In [ ]:
for token in doc:
    print(token.text)

In [ ]:

doc

In [ ]:
for token in doc:
    print(token.text, token.lemma_)

In [ ]:
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
for sent in doc.sents:
    print(sent)

In [ ]:
from spacy import displacy

In [ ]:
displacy.render(doc, style='dep')

In [ ]:
displacy.render(doc, style='dep', options={'compact':True, 'distance': 100})

In [61]:
df['keywords'][0]

[('paramount golf forest', 0.0014224031713185796),
 ('floor paramount golf', 0.0016884701784699123),
 ('well-deserved healthy lifestyle', 0.003589619935348917),
 ('forest ac apartments', 0.008371842743181764),
 ('floor paramount', 0.009914223891687604),
 ('paramount golf', 0.015284005389273585),
 ('golf forest', 0.015284005389273585),
 ('apartments complimented', 0.01636643116542659),
 ('healthy lifestyle', 0.019410041520969947),
 ('green golf', 0.030076153654606908),
 ('complimented by designer', 0.03223943683265993),
 ('designer landscape', 0.03223943683265993),
 ('well-deserved healthy', 0.03223943683265993),
 ('golf', 0.08260749754765566),
 ('paramount', 0.09111716020807922),
 ('forest', 0.09111716020807922),
 ('apartments', 0.09111716020807922),
 ('floor', 0.10773927715390114),
 ('family', 0.10773927715390114),
 ('lifestyle', 0.10773927715390114),
 ('complimented', 0.17672724192998282),
 ('designer', 0.17672724192998282),
 ('landscape', 0.17672724192998282),
 ('green', 0.176727241

In [62]:
def get_keywords(x):
    keywords=[]
    for key,value in x:
        if value < 0.05:
            keywords.append(key)
            
    return keywords
            

In [63]:
df['keywords']=df.keywords.apply(lambda x : get_keywords(x))

In [65]:
df['keywords'][0]

['paramount golf forest',
 'floor paramount golf',
 'well-deserved healthy lifestyle',
 'forest ac apartments',
 'floor paramount',
 'paramount golf',
 'golf forest',
 'apartments complimented',
 'healthy lifestyle',
 'green golf',
 'complimented by designer',
 'designer landscape',
 'well-deserved healthy']

In [66]:
features="I want 2 BHK with study apartment with 1400 sqft between 6 to 12 floor in north-east facing in Greater Noida West"

In [73]:
features=lemmatize(features)

In [74]:
features

'I want 2 BHK study apartment 1400 sqft 6 12 floor north-east facing Greater Noida West'

In [75]:
feature_list=keyword_extractor(features)

In [76]:
feature_list

[('greater noida west', 0.001881309737406442),
 ('bhk study apartment', 0.004390086440759893),
 ('facing greater noida', 0.00688415006041516),
 ('noida west', 0.012602360123953448),
 ('north-east facing greater', 0.015042304599106411),
 ('floor north-east facing', 0.01655915082773619),
 ('greater noida', 0.02140921543860024),
 ('bhk study', 0.026233073037508336),
 ('sqft', 0.04491197687864554),
 ('facing greater', 0.04498862876540802),
 ('study apartment', 0.04940384002065631),
 ('floor north-east', 0.04940384002065631),
 ('bhk', 0.08596317751626563),
 ('west', 0.08596317751626563),
 ('north-east facing', 0.09700399286574239),
 ('greater', 0.1447773057422032),
 ('noida', 0.1447773057422032),
 ('apartment', 0.15831692877998726),
 ('floor', 0.15831692877998726),
 ('study', 0.29736558256021506),
 ('north-east', 0.29736558256021506),
 ('facing', 0.29736558256021506)]

In [77]:
feature_list=get_keywords(feature_list)

In [78]:
feature_list

['greater noida west',
 'bhk study apartment',
 'facing greater noida',
 'noida west',
 'north-east facing greater',
 'floor north-east facing',
 'greater noida',
 'bhk study',
 'sqft',
 'facing greater',
 'study apartment',
 'floor north-east']

In [82]:
list3 = set(df['keywords'][1])&set(feature_list) # we don't need to list3 to actually be a list

list4 = sorted(list3, key = lambda k : df['keywords'][0].index(k))

In [83]:
list3

set()

In [91]:
for x in range(0,len(df['keywords'])):
    if (set(df['keywords'][x]) & set(features)):
        print(df.name)

In [87]:
list3